# Настал этот ужасный момент. Нужно создавать четвёртый файл. Но предобработка НАКОНЕЦ-ТО завершена

как всегда, достанем датасет с гитхаба

In [ ]:
df <- read.csv("https://raw.githubusercontent.com/EmilEybatove/Health/main/helth_after_isolation_forest.csv")

NameError: ignored

Создадим константу, в которой будем хранить все пары лет (как-бы волны)

In [ ]:
years <- paste(as.character(seq(1991, 2011, by=2)), "-", as.character(seq(1992, 2012, by=2)), sep="")

## Полиномиальная модель

Теперь, построим логит. модель, используя предположение о **полиномиальной зависимости** и посмотри на её псевдо-R^2

In [ ]:
df %>%
    group_by(year) %>%
    summarise(R_sqrt = with(summary(glm(
    depres ~ family_problems + ragender + ragey_b + pension + unemployed +
      log_income + log_income2 + raedyrs + disabled, family = "binomial")), 1 - deviance/null.deviance),

    log_income_coef = summary(glm(
    depres ~ family_problems + ragender + ragey_b + pension + unemployed +
      log_income + log_income2 + raedyrs + disabled, family = "binomial"))$coefficients["log_income", "Estimate"],

    log_income_pr = summary(glm(
    depres ~ family_problems + ragender + ragey_b + pension + unemployed +
      log_income + log_income2 + raedyrs + disabled, family = "binomial"))$coefficients["log_income", "Pr(>|z|)"],

    log_income_2_coef = summary(glm(
    depres ~ family_problems + ragender + ragey_b + pension + unemployed +
      log_income + log_income2 + raedyrs + disabled, family = "binomial"))$coefficients["log_income2", "Estimate"],

    log_income_2_pr = summary(glm(
    depres ~ family_problems + ragender + ragey_b + pension + unemployed +
      log_income + log_income2 + raedyrs + disabled, family = "binomial"))$coefficients["log_income2", "Pr(>|z|)"],
)

year,R_sqrt,log_income_coef,log_income_pr,log_income_2_coef,log_income_2_pr
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1991-1992,0.12138224,0.34268994,1.881763e-11,-0.034261238,3.955918e-15
1993-1994,0.12438674,0.32646865,3.198033e-13,-0.032888392,1.538328e-17
1995-1996,0.10308799,0.14924140,1.527977e-03,-0.015015028,2.178708e-04
1997-1998,0.07929740,0.16876540,1.834395e-04,-0.015353026,5.533564e-05
1999-2000,0.07806055,0.21522301,2.095721e-06,-0.021831834,8.722351e-09
2001-2002,0.07177097,0.22402718,1.042903e-05,-0.023193069,2.203824e-08
2003-2004,0.07723664,0.06588256,1.593081e-01,-0.009136962,1.337465e-02
2005-2006,0.08849607,0.16735180,1.092735e-03,-0.018044300,6.383350e-06
2007-2008,0.06551647,0.20467646,1.103537e-04,-0.020341879,8.718067e-07


проведём VIF тест для такой модели. Для этого используем библиотеку car, в которой есть подходящая команда

In [ ]:
# install.packages("car")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘pbkrtest’




In [ ]:
library(car)

Loading required package: carData


Attaching package: ‘car’


The following object is masked from ‘package:dplyr’:

    recode




In [ ]:
df %>%
    group_by(year) %>%
    summarise(VIF = vif(glm(
    depres ~ family_problems + ragender + ragey_b + pension + unemployed +
      log_income + log_income2 + raedyrs + disabled, family = "binomial"))["log_income2"])

year,VIF
<fct>,<dbl>
1991-1992,16.62726
1993-1994,17.73338
1995-1996,18.27456
1997-1998,16.33044
1999-2000,15.83481
2001-2002,14.27132
2003-2004,13.42739
2005-2006,12.86453
2007-2008,12.52493


Есть основания исключить эту переменную из-за очень высокого VIF

## Линейная модель

Теперь построим такую же модель с линейной зависимостью

In [ ]:
df %>%
    group_by(year) %>%
    summarise(R_sqrt = with(summary(glm(
    depres ~ family_problems + ragender + ragey_b + pension + unemployed +
      log_income + log_income2 + raedyrs + disabled, family = "binomial")), 1 - deviance/null.deviance),

    log_income_coef = summary(glm(
    depres ~ family_problems + ragender + ragey_b + pension + unemployed +
      log_income + log_income2 + raedyrs + disabled, family = "binomial"))$coefficients["log_income", "Estimate"],

    log_income_pr = summary(glm(
    depres ~ family_problems + ragender + ragey_b + pension + unemployed +
      log_income + log_income2 + raedyrs + disabled, family = "binomial"))$coefficients["log_income", "Pr(>|z|)"])

year,R_sqrt,log_income_coef,log_income_pr
<fct>,<dbl>,<dbl>,<dbl>
1991-1992,0.12138224,0.34268994,1.881763e-11
1993-1994,0.12438674,0.32646865,3.198033e-13
1995-1996,0.10308799,0.14924140,1.527977e-03
1997-1998,0.07929740,0.16876540,1.834395e-04
1999-2000,0.07806055,0.21522301,2.095721e-06
2001-2002,0.07177097,0.22402718,1.042903e-05
2003-2004,0.07723664,0.06588256,1.593081e-01
2005-2006,0.08849607,0.16735180,1.092735e-03
2007-2008,0.06551647,0.20467646,1.103537e-04


## Модель с использованием инструментов

используем библиотеку ivtools для работы с инструментами

In [ ]:
# install.packages("ivtools")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘nleqslv’, ‘ahaz’




In [ ]:
library(ivtools)

напишем функцию, которая выдаёт модель с инструментом за конкретный год

Пока, не уверен, что работает метод из библиотеки, поэтому пишу свой

In [ ]:
ivmodel2 <- function(year){
  subdf <- df[df$year == year,]
  instrument <- glm(sum ~ ragey_b + age_2 + pension + disabled + raedyrs + unemployed, data=subdf)
  model <- glm(depres ~ family_problems + ragender + I(log(sum)), data=subdf, family="binomial")
  return(summary(model))
  }

In [ ]:
ivmodel <- function(year){
  subdf <- df[df$year == year,]
  fitX.LZ <- glm(sum ~ ragey_b + age_2 + pension + disabled + raedyrs + unemployed, data=subdf)
  fitY.LX <- glm(depres ~ family_problems + ragender + I(log(sum)), data=subdf, family="binomial")
  fitIV <- ivglm(estmethod="ts", fitX.LZ=fitX.LZ, fitY.LX=fitY.LX, data=subdf)
  return(fitIV)
  # return(summary(fitIV))
}

In [ ]:
df %>%
  group_by(year) %>%
  summarise(
    coef = ivmodel2(year)$coefficients["I(log(sum))", "Estimate"],
    prob = ivmodel2(year)$coefficients["I(log(sum))", "Pr(>|z|)"])

Warning message:
“There were 44 warnings in `summarise()`.
The first warning was:
ℹ In argument: `coef = ivmodel2(year)$coefficients["I(log(sum))", "Estimate"]`.
ℹ In group 1: `year = 1991-1992`.
Caused by warning in `==.default`:
! longer object length is not a multiple of shorter object length
ℹ Run `dplyr::last_dplyr_warnings()` to see the 43 remaining warnings.”


year,coef,prob
<fct>,<dbl>,<dbl>
1991-1992,-0.10169398,5.187167e-20
1993-1994,-0.08630215,1.247200e-18
1995-1996,-0.06321774,3.090247e-10
1997-1998,-0.05083703,9.957110e-07
1999-2000,-0.08408349,3.587330e-15
2001-2002,-0.09947572,6.212769e-15
2003-2004,-0.10088466,9.326438e-17
2005-2006,-0.11586385,6.236496e-18
2007-2008,-0.09061435,9.570511e-11
